# Ananke

This notebook is based on the example notebook in py-ananke's repository "https://github.com/athob/py-ananke/tree/main"


In [2]:
import ananke

In [5]:
import numpy as np

In [70]:
import matplotlib.pyplot as plt

In [10]:
import h5py

In [11]:
snapname = "/astro/dboyea/sculptor/orbits/orbit1/1e6_V32_r5.4/out/snapshot_204.hdf5"

In [26]:
starsname = "/astro/dboyea/sculptor/isolation/1e6/fiducial/ana_stars/exp2d_rs0.10_stars.hdf5"

In [21]:
h5f["PartType1"].keys()

<KeysViewHDF5 ['Acceleration', 'Coordinates', 'ExtPotential', 'ParticleIDs', 'Potential', 'Velocities']>

In [83]:
ananke.Ananke.make_dummy_particles_input()

{'pos3': array([[-20.4573609 , -17.48192114, -21.13305532],
        [ 51.38594388, -13.14913922,  62.77950588],
        [ -1.55235669, -33.82605987, -42.82451732],
        ...,
        [ 29.71119203, -14.70307752,  -7.81244932],
        [ 36.03981695,   6.66570441,  25.10326859],
        [ 44.46333134,   8.11340193, -37.01696902]]),
 'vel3': array([[107.78552251, 106.17435395,  45.67363079],
        [ 18.53149118,  17.55029679, -98.5022028 ],
        [ 53.23967913, -31.03603957,  39.21340481],
        ...,
        [ 30.56127375, -56.12446118,  12.18744563],
        [ 34.86748772,  16.88226449,  11.34672757],
        [ 57.76330936, -84.75913509,  20.32379158]]),
 'mass': array([5760.04811654, 4576.05212445, 4796.15621176, ..., 4233.2813766 ,
        5315.2200421 , 5511.70849731]),
 'age': array([10.15190437,  9.84097084,  9.4860017 , ...,  9.85498871,
         9.85129939,  9.47178362]),
 'feh': array([-0.58024846, -0.70340326, -0.90510346, ..., -0.86874064,
        -0.14913917, -0.98961

In [25]:
h5f = h5py.File(snapname)
pos = np.array(h5f["PartType1/Coordinates"])
vel = np.array(h5f["PartType1/Velocities"])
index =  np.array(h5f["PartType1/ParticleIDs"])
h5f.close()

In [32]:
h5f = h5py.File(starsname)
stars_index = np.array(h5f["index"])
probabilities = np.array(h5f["probability"])
h5f.close()

stars_index, probabilities

(array([      1,       2,       3, ...,  999998,  999999, 1000000]),
 array([2.22934815e-251, 0.00000000e+000, 0.00000000e+000, ...,
        4.10473853e-152, 4.17184933e-020, 3.90043577e-122]))

In [37]:
mtot = 5e6
mstar = probabilities[np.argsort(index)] * mtot

In [38]:
sum(mstar)

4999999.999999744

In [88]:
N = len(mstar)
sim_input = {
    "pos3": pos,
    "vel3": vel * V0,
    "mass": mstar,
    "age": np.random.normal(9.8, 0.1, N),
    "feh": np.random.normal(-2.3, 0.25, N),
    "parentid": index,
    
}

In [89]:
sim_input

{'pos3': array([[ 16.794956 ,  67.28466  , -93.30016  ],
        [ 17.47276  ,  69.20219  , -91.83909  ],
        [ 14.343069 ,  67.29755  , -92.37616  ],
        ...,
        [ -0.7272042,  10.73672  ,   7.5973988],
        [ -0.7468634,  10.044104 ,   7.2588286],
        [  0.5273622,  10.707563 ,   7.4367175]], dtype=float32),
 'vel3': array([[ 1.3682275e+01,  6.6727104e+01,  8.3488213e+01],
        [ 1.0661245e+01,  8.9569786e+01,  8.8142433e+01],
        [ 2.4167257e+01,  8.2789963e+01,  7.7108398e+01],
        ...,
        [ 4.9446499e+01, -2.9987799e+02,  2.8754111e+02],
        [-1.6626580e-01,  1.4960247e+01,  1.7744212e+02],
        [-7.6191578e+00, -1.3778018e+02,  3.3656036e+02]], dtype=float32),
 'mass': array([1.11319416e-029, 4.97268723e-029, 2.64274705e-155, ...,
        0.00000000e+000, 0.00000000e+000, 1.35403117e-184]),
 'age': array([9.5632495 , 9.90458193, 9.74931422, ..., 9.86364106, 9.70299866,
        9.98642849]),
 'feh': array([-2.51933998, -1.97159863, -2.456

In [90]:
V0 = 207 # km/s

In [91]:
help(ananke.Ananke)

Help on class Ananke in module ananke.Ananke:

class Ananke(builtins.object)
 |  Ananke(particles: Dict[str, numpy.ndarray[Any, numpy.dtype[+_ScalarType_co]]], name: str, ngb: int = 64, d_params: Dict[str, Any] = {}, e_params: Dict[str, Any] = {}, err_params: Dict[str, Any] = {}, **kwargs: Dict[str, Any]) -> None
 |  
 |  Represents a single ananke pipeline.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, particles: Dict[str, numpy.ndarray[Any, numpy.dtype[+_ScalarType_co]]], name: str, ngb: int = 64, d_params: Dict[str, Any] = {}, e_params: Dict[str, Any] = {}, err_params: Dict[str, Any] = {}, **kwargs: Dict[str, Any]) -> None
 |      Parameters
 |      ----------
 |      particles : dict
 |          A dictionary of same-length arrays representing particles
 |          data of a stellar population - see notes for formatting
 |      
 |      name : str
 |          Name for the pipeline
 |      
 |      ngb : int
 |          Number of neighbours to use in kernel density estimatio

In [92]:
observer = {
    "pos3": [8.122, 0, 0.0208],
    "vel3": [ 12.9, 245.6,    7.78],
}

rshell = [0, 1000]

fsample = 0.01

photo_sys = 'padova/GAIA__DR2'

cmd_magnames = {'magnitude': 'G',
                'color_minuend': 'Gbp',
                'color_subtrahend': 'Grp'}

cmd_box = {
           'abs_mag_lim_lo': -1000,
           'abs_mag_lim_hi': 1000,
        #    'app_mag_lim_lo' : -1000,
           'app_mag_lim_hi': 30,
        #    'color_lim_lo' : -1000,
        #    'color_lim_hi' : 1000
           }

In [93]:
name = "fiducial"

an = ananke.Ananke(sim_input, name, 
    fsample=fsample, observe=observer, rshell=rshell, photo_sys=photo_sys, cmd_magnames=cmd_magnames, **cmd_box)


/astro/dboyea/.local/lib/python3.9/site-packages/ananke/utils.py:26: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self._record_of_all_used_keys = set()
/astro/dboyea/.local/lib/python3.9/site-packages/ananke/utils.py:26: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self._record_of_all_used_keys = set()


In [ ]:
survey = an.run()

In [ ]:
survey

In [ ]:
observable = survey[~survey.dec.isna()]
observable

In [ ]:
plt.scatter(observable.ra.values, observable.dec.values, alpha=0.1)